In [ ]:
# set up and test simple commands with LLM 
# swaped OpenAI for Mistral due to expired free tier quota

In [ ]:
# import mistral 
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [ ]:
import os

In [ ]:
# get API key stored in env variable
api_key = os.getenv("MISTRAL_KEY")

In [ ]:
client = MistralClient(api_key = api_key)

In [ ]:
# ask it a question
response = client.chat(
    model = 'mistral-tiny',
    messages = [ChatMessage(role = "user", content = "Is it too late to join the course?")]
)

In [ ]:
print(response.choices[0].message.content)

In [ ]:
# set up a search using minisearch from Alexei Grigorev
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [ ]:
import minsearch

In [ ]:
import json

In [ ]:
# get the document
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [ ]:
# index (fit the index to) the documents
index.fit(documents)

In [ ]:
# some query (question) 
query = 'the course has already started, can I still enroll?'

In [ ]:
# let's search but make some fields more important and some less so
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    query,
    boost_dict = boost, 
    # filter_dict = {'course': 'data-engineering-zoomcamp'}, # if we want to search only for specific course 
    num_results = 5
)

In [ ]:
print(results)

In [ ]:
# combine LLM and search

In [ ]:
# create the prompt template
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [ ]:
# define the context
context = ""
for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
prompt = prompt_template.format(question = query, context = context).strip()

In [ ]:
print(prompt)

In [ ]:
# feed it to LLM
response = client.chat(
    model = "mistral-large-latest",
    messages = [ChatMessage(role = "user", content = prompt)]
)
print(response.choices[0].message.content)

In [ ]:
# clean up the flow

In [ ]:
# define a function for search
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    search_results = index.search(
        query,
        boost_dict = boost, 
        # filter_dict = {'course': 'data-engineering-zoomcamp'}, # if we want to ask only about one specific course 
        num_results = 10
    )
    return search_results
    

In [ ]:
# define a function for building a prompt
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question = query, context = context).strip()
    return prompt
    

In [ ]:
# define a function to talk to LLM
def llm(prompt):
    response = client.chat(
    model = "mistral-large-latest",
    messages = [ChatMessage(role = "user", content = prompt)]
    )    
    return response.choices[0].message.content

In [ ]:
# call these functions within in one other function
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
print(rag('the course has already started, can I enroll now?'))

In [ ]:
# replace Alexey's search with elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
# create elastic search client
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
# check that it worked and retrieve info for hw1 Q1
es_client.info()['version']['build_hash']

In [ ]:
# create an index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # we use course names for filtering
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index = index_name, body = index_settings)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# index the docs
for doc in documents:
    es_client.index(index = index_name, document = doc)

In [ ]:
query = 'I just discovered the course. Can I still join it?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # ^3 - makes this field 3x important
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return result_docs

In [ ]:
search_results = elastic_search(query)
print(search_results)

In [ ]:
# adjust rag function
def rag_es(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
rag_es("The course has already started, can I still join?")

In [ ]:
# TODO: HW